In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.python.keras.callbacks import TensorBoard
from time import time

In [2]:
spec_df = pd.read_csv("resources/train_csv/dataset_800_each.csv")

In [3]:
spec_df.head()

,labels,p0,p1,p2,p3,p4,p5,p6,p7,p8,...,p9149,p9150,p9151,p9152,p9153,p9154,p9155,p9156,p9157,p9158
0,6,0.024392,0.001553,0.009607,0.000859,0.003031,0.008380,0.000226,0.001299,0.000530,...,2.157441e-07,5.489773e-06,1.872812e-06,1.068106e-06,2.203698e-06,5.016204e-06,6.863662e-06,0.000004,2.353581e-06,4.129150e-06
1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.966977e-05,6.467756e-07,6.708343e-06,1.318529e-06,4.921577e-07,1.183479e-07,1.977154e-06,0.000004,1.446733e-07,9.138176e-07
2,4,0.000185,0.006543,0.072075,0.173826,46.050701,0.000073,12.460474,316.095428,180.521927,...,1.246565e-05,9.886509e-06,3.520680e-06,3.635183e-07,8.782088e-07,1.139976e-06,2.753889e-06,0.000004,2.526484e-06,1.108028e-05
3,6,0.000495,0.000325,0.000035,0.000025,0.000003,0.000028,0.000036,0.000002,0.000009,...,1.953603e-06,1.630203e-05,1.214632e-05,2.255445e-07,3.352303e-05,1.039845e-05,6.236378e-06,0.000004,1.455079e-05,5.294188e-06
4,7,0.001554,0.000647,0.000515,6.112334,18.013880,3.559748,52.586349,0.006034,4.393357,...,2.110135e-06,8.222524e-06,1.129225e-07,1.395569e-05,1.340832e-05,1.033570e-06,5.042800e-10,0.000002,1.438515e-05,3.324445e-06


In [14]:
tensorboard = TensorBoard(log_dir="logs/model_1_{}".format(time()))

In [5]:
labels = spec_df['labels'].values
labels = tf.keras.utils.to_categorical(labels) # one hot encode labels
spec_df.drop(labels="labels", axis='columns', inplace=True)
features = spec_df.values
features = features.reshape(-1, 129, 71, 1)

In [15]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=(1, 1), padding="same",
                                 activation='relu', input_shape=(129, 71, 1)))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=(1, 1), padding="same", activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(5, 5), padding="same"))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding="same", activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding="same", activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3), padding="same"))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.03, beta_1=0.9, beta_2=0.999)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 129, 71, 32)       832       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 129, 71, 32)       25632     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 26, 15, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 26, 15, 64)        18496     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 26, 15, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 9, 5, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2880)             

In [16]:
model.fit(x=features,
          y=labels,
          batch_size=128,
          epochs=4,
          validation_split=0.2,
          callbacks=[tensorboard])

Train on 5600 samples, validate on 2400 samples
Epoch 1/4
5600/5600 [==============================] - 512s 91ms/sample - loss: 694.1984 - accuracy: 0.0923 - val_loss: 2.3044 - val_accuracy: 0.1025
Epoch 2/4
5600/5600 [==============================] - 511s 91ms/sample - loss: 2.3033 - accuracy: 0.1025 - val_loss: 2.3038 - val_accuracy: 0.0921
Epoch 3/4
1280/5600 [=====>........................] - ETA: 6:08 - loss: 2.3025 - accuracy: 0.0844

KeyboardInterrupt: 